This notebook will be collected automatically at **6pm on Monday** from `/home/data_scientist/assignments/Week11` directory on the course JupyterHub server. If you work on this assignment on the course Jupyterhub server, just make sure that you save your work and instructors will pull your notebooks automatically after the deadline. If you work on this assignment locally, the only way to submit assignments is via Jupyterhub, and you have to place the notebook file in the correct directory with the correct file name before the deadline.

1. Make sure everything runs as expected. First, restart the kernel (in the menubar, select `Kernel` → `Restart`) and then run all cells (in the menubar, select `Cell` → `Run All`).
2. Make sure you fill in any place that says `YOUR CODE HERE`. Do not write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed by the autograder.
3. Do not change the file path or the file name of this notebook.
4. Make sure that you save your work (in the menubar, select `File` → `Save and CheckPoint`)

## Problem 2. Hierarchical Modeling.

In this problem, we will implement a hierarchical model to estimate the distribution of departure delays in December of 2001 ([airline on-time performance data](http://stat-computing.org/dataexpo/2009/the-data.html).)

![](https://github.com/UI-DataScience/accy570-fa16/raw/master/Week14/images/hierarchical.png)

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm

from nose.tools import assert_equal, assert_true, assert_is_instance
from numpy.testing import assert_array_almost_equal, assert_almost_equal

We use the [airline on-time performance data](http://stat-computing.org/dataexpo/2009/). For simplicity, we limit our analysis to flights that departed from [CMI](https://en.wikipedia.org/wiki/University_of_Illinois_Willard_Airport) in December.

In [ ]:
filename = '/home/data_scientist/data/2001.csv'

usecols = (1, 2, 15, 16)
columns = ['Month', 'DayofMonth', 'DepDelay', 'Origin']

all_data = pd.read_csv(filename, header=0, na_values=['NA'], usecols=usecols, names=columns)

local = all_data[
    (all_data['Origin'] == 'CMI') & # use only flights departed from Chicago
    (all_data['Month'] == 12) # consider only December
    ]
local = local.drop(['Month', 'Origin'], axis=1) # we don't need Month and Origin columns
local = local.dropna() # drop missing values

print(local.head())

We will use a [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) (see the next section) but the Poisson distribution does not allow negative means. That is, $\mu > 0$ in

$$ Poisson(\mu) = P(x\mid\mu) = \frac{e^{-\mu}\mu^{x}}{x!}\quad\textrm{for}\, x=0,1,2,\cdots $$

However, there exist some negative values in `DepDelay`, as shown in the following code cell.

In [ ]:
print(local.DepDelay.min())

This condition can be avoided by a simple shift in the domain, so let's add 15 minutes to all departure delays.

In [ ]:
def shift_column(df, field, shift):
    return pd.concat([df.drop(field, axis=1), df[field].apply(lambda x: x + shift)], axis=1) 

local_shifted = shift_column(local, 'DepDelay', 15)

print(local_shifted.head())

The following code cell asserts that all values in `DepDelay` are non-negative.

In [ ]:
assert_equal((local_shifted.DepDelay.values < 0).sum(), 0)

For simplicity, let's remove some outliers and only consider departure delays less than 60 minutes.

In [ ]:
local_shifted = local_shifted[local_shifted['DepDelay'] < 60]
# check if there are any values greater than 60
assert_equal((local_shifted.DepDelay.values > 60).sum(), 0)

In the following section, we model each day independently, modeling paramters $\mu_i$ of the Poisson distribution for each day of December, $i=1, 2, \cdots, 31$. The reasoning behind this is that the departure delays will depend on different conditions of each day, e.g. the weather, whether it's a weekend or a weekday, whehter it's a holiday, etc.

Simiarly to the use of `county_idx` in one of the [required readings](https://github.com/UI-DataScience/accy570-fa16/blob/master/Week14/lesson2.md), we need a way to map `mu` (an array of length 31) to an array that has the length as `local_shifted`. Read [GLM: Hierarchical Linear Regression](https://pymc-devs.github.io/pymc3/notebooks/GLM-hierarchical.html) to see how `county_idx` is used [here](https://pymc-devs.github.io/pymc3/notebooks/GLM-hierarchical.html#Hierarchical-Model) and how `participants_idx` is used [here](http://nbviewer.jupyter.org/github/markdregan/Bayesian-Modelling-in-Python/blob/master/Section%203.%20Hierarchical%20modelling.ipynb). 

We can use the `DayofMonth` column to create `date_idx`:

In [ ]:
date_idx = local_shifted['DayofMonth'].values - 1
print(date_idx)

And we can use `date_idx` as follows:

```python
>>> mu = np.arange(31)
>>> print(mu)
```
```
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
```
```python
>>> print(mu[date_idx])
```
```
[ 0  1  2  3  4  5  6  8  9 10 11 12 13 14 15 17 18 19 21 22 23 24 25 26 27
 28 29 30  0  1  2  3  4  5  6  7  8 10 12 14 16 17 18 20 25 26 28 29 30  0
  1  3  4  5  6  7  8  9 10 12 14 15 16 18 20 21 22 24 26 27 28 29 30  2  3
  4  5  6  8  9 10 11 12 13 14 16 17 18 19 20 21 22 23 24 25 27 28 29 30  0
  2  3  4  8  9 10 11 13 15 17 18 19 20 21 22 23 24 25 26 27 29]
```
```python
>>> len(mu[date_idx]) == len(local_shifted)
```
```
True
```

## Use PyMC3 to implement a hierarchical model

- Implement the following hierarchical model using `pymc3`:
$$
\begin{aligned}
y_{ji} &\sim Poisson(\mu_{i}) \\
\mu_i &= Gamma(\alpha=\alpha_\mu, \beta=\beta_\mu) \\
\alpha_\mu &= Gamma(\alpha=1, \beta=1) \\
\beta_\mu &= Gamma(\alpha=1, \beta=1)
\end{aligned}
$$
for each flight $j$ and each day $i$. Here, $Gamma(\alpha, \beta)$ is the [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution).

- Use the names `mu`, `hyper_alpha_mu`, and `hyper_beta_mu` for $\mu_i$, $\alpha_\mu$, and $\beta_\mu$, respectively.


Hints:

- The bulk of the code is provided. Fill in the spaces between
```python
with pm.Model() as model:
```
and
```python
    y_est = pm.Poisson('y_est', mu=mu[idx], observed=X)
```

- You basically need to write only 3 lines:
  1. Define `hyper_alpha_mu`,
  2. Define `hyper_beta_mu`, and
  3. Define `mu`.
  
- Specify the `shape` paramter in `mu`. The shape of `mu` should be equal to `n_days` because we have 31 days and we sample the distribution for each day. For example, your definition of `mu` should look something like
  ```python
  mu = pm.Gamma(..., shape=n_days)
  ```
  (The `...` symbol in the above example is not code. You have fill this part in.)

- Note there are two ways to specify a [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution): either in terms of alpha and beta or mean and standard deviation. In this problem, we parametrize it in terms of alpha (the shape parameter) and beta (the rate parameter). Thus, you should use the optional paramters, `alpha` and `beta`, in [pymc3.Gamma()](https://pymc-devs.github.io/pymc3/api.html#pymc3.distributions.continuous.Gamma).

- `pymc3.find_MAP()` estimates the model paramters with the [maximum a posteriori (MAP) method](https://en.wikipedia.org/wiki/Maximum_a_posteriori_estimation).

- `pymc3.Metropolis()` uses the [Metropolis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) to generate posterior samples.

- Note that for reproducibility we use the `random_seed` parameter in `pymc3.sample()`.

In [ ]:
def sample_posterior(X, idx, n_samples=2000, random_seed=0):
    """
    A hierarchical Poisson model.
    
    Paramters
    ---------
    X: A numpy array
    y: A numpy array
    n_samples: The number of samples to draw in pymc3.sample().
               Defaults to 2000.
    random_seed: An int. Used in pymc3.sample().
                 Defaults to 0.
                 
    Returns
    -------
    A pymc3.MultiTrace object with access to sampling values.
    """
    
    n_days = 31
    
    with pm.Model() as model:
        
        # YOUR CODE HERE

        y_est = pm.Poisson('y_est', mu=mu[idx], observed=X)
        
        y_pred = pm.Poisson('y_pred', mu=mu[idx], shape=len(X))
    
        start = pm.find_MAP()
        step = pm.Metropolis(start=start)
        trace = pm.sample(n_samples, step=step, start=start, random_seed=random_seed)
    
    return trace

In [ ]:
hierarchical_trace = sample_posterior(X=local_shifted['DepDelay'].values, idx=date_idx, n_samples=80000, random_seed=0)

In [ ]:
assert_is_instance(hierarchical_trace, pm.backends.base.MultiTrace)

assert_true('mu' in hierarchical_trace.varnames)
assert_true('hyper_alpha_mu' in hierarchical_trace.varnames)
assert_true('hyper_beta_mu' in hierarchical_trace.varnames)

for v in hierarchical_trace.varnames:
    assert_equal(len(hierarchical_trace[v]), 80000)
    
assert_equal(hierarchical_trace['mu'].shape[1], 31)

# note the length of array is 31 for 31 days in the month
assert_array_almost_equal(
    hierarchical_trace['mu'][0],
    [ 5.15005593,  16.62475093,   5.03444126,   5.40358022,
      8.40843764,   5.61251444,   9.65902678,   8.29027819,
      5.77918745,   7.8091926 ,   6.7182054 ,  11.21074812,
     11.27763162,   7.45102393,  13.82115361,  12.86502678,
      9.40608048,   7.23111944,  12.63401805,  13.31619368,
     13.35869547,   6.9998901 ,   5.72812911,   5.64635637,
      4.68759738,   8.8497243 ,  10.81517309,   7.23111939,
      4.57198275,   9.06575018,   4.57198268]
   )
assert_almost_equal(hierarchical_trace['hyper_alpha_mu'][0], 5.5450171527810559)
assert_almost_equal(hierarchical_trace['hyper_beta_mu'][0], 0.64942393608716098)

assert_array_almost_equal(
    hierarchical_trace['mu'][-1],
    [ 8.08654951,  34.0837555 ,   7.57008046,  10.5119628 ,
     18.50107315,  11.78745223,  18.09940974,  16.12241455,
     12.13169849,  12.99963025,  13.07349382,  21.31053708,
     27.47656623,  11.39435595,  28.41184314,  22.42567766,
     16.35230353,  12.67814566,  26.50712576,  24.24614846,
     21.82844008,  15.66077269,   7.92093998,  10.11164246,
      8.82131075,  19.24463488,  23.58609882,  13.27383155,
      8.12750503,  16.63355873,   7.75384526]
     )
assert_array_almost_equal(
    hierarchical_trace['hyper_alpha_mu'][-5:],
    [ 4.80090357,  4.9096912 ,  4.92489959,  4.63708045,  4.63708045]
)
assert_array_almost_equal(
    hierarchical_trace['hyper_beta_mu'][-5:],
    [ 0.29688427,  0.29688427,  0.29688427,  0.29688427,  0.29688427]
)

Let's check our model by inspecting the sampling output. A simple posterior plot can be created using `traceplot()`.

In [ ]:
pm.traceplot(hierarchical_trace[40000:], varnames=['mu', 'hyper_alpha_mu', 'hyper_beta_mu']);

Finally, we can visually inspect the fit of our model by comparing the posterior predictive distribution and the distribution of the observed data

In [ ]:
x_lim = 60
n_burn = 40000

# we discard burn-in and use every 1000th trace
y_pred = hierarchical_trace.get_values('y_pred')[n_burn::1000].ravel()

sns.set_style('white')
fig, ax = plt.subplots(2, sharex=True, figsize=(12,6))

ax[0].hist(y_pred, range=[0, x_lim], bins=x_lim, histtype='stepfilled')   
ax[0].set_xlim(1, x_lim)
ax[0].set_ylabel('Frequency')
ax[0].set_title('Posterior predictive distribution')

ax[1].hist(local_shifted.DepDelay.values, range=[0, x_lim], bins=x_lim, histtype='stepfilled')
ax[1].set_xlabel('Departure Delay + 15 minutes')
ax[1].set_ylabel('Frequency')
ax[1].set_title('Distribution of observed data')

sns.despine()

plt.tight_layout()

plt.show()